In [1]:
import pandas as pd
import numpy as np
import math
import glob
import os

In [2]:
parent_folders = ['/home/bvelasco/CovidVax_DM/Results/binary_eof/grace_period_180/*/*.csv',
                 '/home/bvelasco/CovidVax_DM/Results/binary_eof/grace_period_360/*/*.csv',
                 '/home/bvelasco/CovidVax_DM/Results/survival/grace_period_180/*/*.csv',
                 '/home/bvelasco/CovidVax_DM/Results/survival/grace_period_360/*/*.csv']

In [3]:
data=None

for _, folder in enumerate(parent_folders):
    
    result_files = glob.glob(folder)
    
    for index, file  in enumerate(result_files):

        if index==0:
            data = pd.read_csv(file)
            data.drop(['Unnamed: 0'], inplace=True, axis=1)
        else:
            newdata = pd.read_csv(file)
            newdata.drop(['Unnamed: 0', 'Intervention'], inplace=True, axis=1)
            data = pd.concat([data, newdata], axis=1)
        
    if  'binary_eof' in folder:
        result_data = pd.concat([data['g-form mean (NICE)'].mean(axis=1),
        data['g-form mean (NICE)'].mean(axis=1)-1.96*data['g-form mean (NICE)'].std(axis=1), 
        data['g-form mean (NICE)'].mean(axis=1)+1.96*data['g-form mean (NICE)'].std(axis=1),
        data['Mean ratio'].mean(axis=1),
        data['Mean ratio'].mean(axis=1)-1.96*data['Mean ratio'].std(axis=1),
        data['Mean ratio'].mean(axis=1)+1.96*data['Mean ratio'].std(axis=1)], axis=1)
    
        result_data=result_data.set_axis(['Probability', '95% LCI Probability', '95% UCI Probability', 'Probability ratio', '95% LCI PR', '95% UCI PR'], axis=1)

    elif 'survival' in folder:
        result_data = pd.concat([data['g-form risk (NICE-parametric)'].mean(axis=1),
        data['g-form risk (NICE-parametric)'].mean(axis=1)-1.96*data['g-form risk (NICE-parametric)'].std(axis=1),
        data['g-form risk (NICE-parametric)'].mean(axis=1)+1.96*data['g-form risk (NICE-parametric)'].std(axis=1),
        data['Risk Ratio'].mean(axis=1),
        data['Risk Ratio'].mean(axis=1)-1.96*data['Risk Ratio'].std(axis=1),
        data['Risk Ratio'].mean(axis=1)+1.96*data['Risk Ratio'].std(axis=1)], axis=1)
    
        result_data=result_data.set_axis(['Survival', '95% LCI Survival', '95% UCI Survival', 'Survival ratio', '95% LCI SR', '95% UCI SR'], axis=1)
    
    result_data = result_data.set_axis(data['Intervention'], axis=0)
    result_data.to_csv(folder.split('*')[0] + 'aggregated_results.csv')